# Predicting F1 Qualification and Race Outcomes Using Practice Session Data

# Introduction
Formula 1 is a sport where milliseconds matter, and predictive insights can be the difference between victory and defeat. This project leverages data from Free Practice sessions (FP1, FP2, and FP3) to predict the outcome of race day—specifically, the eventual race winner. From a motorsport strategy perspective, we analyze key performance indicators such as lap times, tire compound choices, stint durations, and tire degradation rates to assess how teams and drivers prepare for qualifying and race simulations.

On the data science front, we build a structured pipeline using the FastF1 library to extract, process, and engineer features from historical Grand Prix data. Machine learning models—including classification and regression techniques—are employed to correlate practice performance with final race outcomes. By combining domain-specific race knowledge with predictive analytics, this project aims to anticipate on-track results before the lights go out.

### Importing Libraries

In [1]:
import fastf1
from fastf1.ergast import Ergast
from fastf1 import plotting
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.decomposition import PCA
import shap
import streamlit as st
import warnings

warnings.filterwarnings('ignore')
fastf1.Cache.enable_cache('./fastf1_cache')

# Load Data using legacy Ergast fallback to avoid SSL verification issues
# import urllib3
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
# import fastf1.ergast.interface as ergast_interface

C:\Users\aeropc6\.conda\envs\dev_env\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\aeropc6\.conda\envs\dev_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load and Extracxt Data

#### Load Session Data

In [ ]:
# Load Session Data
def load_session_data(season, gp, session):
    ses = fastf1.get_session(season, gp, session)
    ses.load()
    return ses

#### Extracting Practice Data

In [2]:
# Extracting Practice Session Stats
def get_practice_data(seasons):
    sessions=['FP1', 'FP2', 'FP3']
    data = []
    for season in seasons:
        print(f"Season: {season}")
        schedule = fastf1.get_event_schedule(season)
        for _, row in schedule.iterrows():
            race = row['EventName']
            if race == 'Pre-Season Test':
                continue
            driver_data = defaultdict(lambda: {
                'LapTimes': [],
                'DriverID': None
            })
            for i, session in enumerate(sessions):
                try:
                    practice_sess = fastf1.get_session(season, race, session)
                    practice_sess.load()

                    for driver in practice_sess.drivers:
                        driver_laps = practice_sess.laps.pick_driver(driver).copy()
                        driver_laps = driver_laps[driver_laps['LapTime'].notna()]
                        
                        if driver_laps.empty:
                            continue
                        lap_times = driver_laps['LapTime'].dt.total_seconds().tolist()
    
                        driver_data[driver]['LapTimes'].extend(lap_times)
                        driver_data[driver]['DriverID'] = practice_sess.get_driver(driver)['DriverId'] 
                except Exception as e:
                    print(f"Skipped {season} {race} {session} due to: {e}")
                    continue

            for driver, stats in driver_data.items():
                if not stats['LapTimes']:
                    continue
                fastest_lap = min(stats['LapTimes'])
                avg_lap = sum(stats['LapTimes'])/len(stats['LapTimes'])
                data.append({
                    'Season': season,
                    'Race': race,
                    'DriverID': stats['DriverID'],
                    f'FastestPracticeLap': fastest_lap,
                    f'AvgPracticeLap': avg_lap,
                    # **tire_compound_counts # e.g. 'Soft': 6, 'Medium': 3
                })

    return pd.DataFrame(data)

In [13]:
seasons = [2021, 2022, 2023, 2024]
practice_df =  get_practice_data(seasons)
practice_df.to_csv('practice_data.csv', index=False)

Season: 2021


events      WARNING 	Correcting user input 'Pre-Season Test' to 'United States Grand Prix'
core           INFO 	Loading data for United States Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


Race: Pre-Season Test


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['3', '4', '5', '6', '7', '9', '10', '11', '14', '16', '18', '22', '31', '33', '44', '47', '55', '63', '77', '99']
events      WARNING 	Correcting user input 'Pre-Season Test' to 'United States Grand Prix'
core           INFO 	Loading data for United States Grand Prix - Practice 2 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req           

Race: Bahrain Grand Prix


...

req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '10', '11', '14', '16', '18', '20', '22', '23', '24', '27', '30', '31', '43', '44', '55', '63', '77', '81']
core           INFO 	Loading data for Las Vegas Grand Prix - Practice 2 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_d

Race: Qatar Grand Prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '10', '11', '14', '16', '18', '20', '22', '23', '24', '27', '30', '31', '43', '44', '55', '63', '77', '81']
events      WARNING 	Correcting user input 'Qatar Grand Prix' to 'Qatar Grand Prix'
events      WARNING 	Correcting user input 'Qatar Grand Prix' to 'Qatar Grand Prix'
core           INFO 	Loading data for Abu Dhabi Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for 

Skipped 2024 Qatar Grand Prix FP2 due to: Session type 'FP2' does not exist for this event
Skipped 2024 Qatar Grand Prix FP3 due to: Session type 'FP3' does not exist for this event
Race: Abu Dhabi Grand Prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '10', '11', '14', '16', '20', '24', '27', '28', '30', '34', '37', '39', '40', '43', '44', '46', '61', '63', '77']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Practice 2 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_

In [11]:
practice_df = pd.read_csv('practice_data.csv', delimiter=',')
practice_df.head()

,Season,Race,DriverID,FastestPracticeLap,AvgPracticeLap
0,2021,Pre-Season Test,ricciardo,95.345,109.992026
1,2021,Pre-Season Test,norris,94.945,107.691345
2,2021,Pre-Season Test,vettel,95.851,106.818941
3,2021,Pre-Season Test,latifi,96.118,109.432850
4,2021,Pre-Season Test,raikkonen,96.062,112.151591


#### Extracting Qualifying Data

In [3]:
def get_qualifying_data(seasons):
    session = 'Q'
    data = []
    for season in seasons:
        print(f"Season: {season}")
        schedule = fastf1.get_event_schedule(season)
        for _, row in schedule.iterrows():
            race = row['EventName']
            if race == 'Pre-Season Test':
                continue
            print(f"Race: {race}")
            try:
                quali_sess = fastf1.get_session(season, race, session)
                quali_sess.load()
            except Exception as e:
                print(f"Skipped {season} {race} {session} due to: {e}")
                continue
                
            results = quali_sess.results
        
            for driver_id, row_data in results.iterrows():
                data.append({
                    'Season': season,
                    'Race': race,
                    'DriverID': row_data['DriverId'],
                    'Q1Time': row_data['Q1'].total_seconds() if pd.notna(row_data['Q1']) else None,
                    'Q2Time': row_data['Q2'].total_seconds() if pd.notna(row_data['Q2']) else None,
                    'Q3Time': row_data['Q3'].total_seconds() if pd.notna(row_data['Q3']) else None,
                    'GridPos': int(row_data['Position']) if pd.notna(row_data['Position']) else None
                })
    
        
    return pd.DataFrame(data)

In [12]:
quali_df = get_qualifying_data(seasons)
quali_df.to_csv('qualifying_data.csv', index=False)
quali_df

Season: 2021


events      WARNING 	Correcting user input 'Pre-Season Test' to 'United States Grand Prix'
core           INFO 	Loading data for United States Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Race: Pre-Season Test


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['33', '44', '11', '77', '16', '55', '3', '4', '10', '22', '31', '5', '99', '14', '63', '18', '6', '7', '47', '9']
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Race: Bahrain Grand Prix


...

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['63', '55', '10', '16', '1', '4', '22', '81', '27', '44', '31', '20', '24', '43', '30', '11', '14', '23', '77', '18']
events      WARNING 	Correcting user input 'Qatar Grand Prix' to 'Qatar Grand Prix'
core           INFO 	Loading data for Qatar Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


Race: Qatar Grand Prix


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '63', '4', '81', '16', '44', '55', '14', '11', '20', '10', '24', '77', '22', '18', '23', '30', '27', '43', '31']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


Race: Abu Dhabi Grand Prix


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '55', '27', '1', '10', '63', '14', '77', '11', '22', '30', '18', '16', '20', '23', '24', '44', '43', '61']


In [10]:
quali_df = pd.read_csv('qualifying_data.csv', delimiter=',')
quali_df.head()

,Season,Race,DriverID,Q1Time,Q2Time,Q3Time,GridPos
0,2021,Pre-Season Test,max_verstappen,94.352,93.464,92.910,1.0
1,2021,Pre-Season Test,hamilton,94.579,93.797,93.119,2.0
2,2021,Pre-Season Test,perez,94.369,94.178,93.134,3.0
3,2021,Pre-Season Test,bottas,94.590,93.959,93.475,4.0
4,2021,Pre-Season Test,leclerc,94.153,93.928,93.606,5.0


#### Extracting Race Result Label Data

In [21]:
def get_race_labels(seasons):
    data = []
    for season in seasons:
        schedule = fastf1.get_event_schedule(season)
        for _, row in schedule.iterrows():
            race = row['EventName']
            if race == 'Pre-Season Test':
                continue
            try:
                race_sess = fastf1.get_session(season, race, 'R')
                race_sess.load()
            except Exception as e:
                    print(f"Skipped {season} {race} {session} due to: {e}")
                
            results = race_sess.results.rename(
                columns={'Position': 'RaceResult'}
            )
            for driver_id, row_data in results.iterrows():
                data.append({
                    'Season': season,
                    'Race': race,
                    'DriverID': row_data['DriverId'],
                    'RacePosition': int(row_data['RaceResult']) if pd.notna(row_data['RaceResult']) else None
                })
        
    return pd.DataFrame(data)

In [25]:
seasons = [2021, 2022, 2023, 2024]
race_df = get_race_labels(seasons)
race_df.to_csv('race_data.csv', index=False)
race_df

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 44 completed the race distance 00:00.067000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['44', '33', '77', '4', '11', '16', '3', '55', '22', '18', '7', '99

KeyboardInterrupt: 

In [26]:
race_df = pd.read_csv('race_data.csv', delimiter=',')
race_df.head()

,Season,Race,DriverID,RacePosition
0,2021,Bahrain Grand Prix,hamilton,1.0
1,2021,Bahrain Grand Prix,max_verstappen,2.0
2,2021,Bahrain Grand Prix,bottas,3.0
3,2021,Bahrain Grand Prix,norris,4.0
4,2021,Bahrain Grand Prix,perez,5.0


#### Merge All Features + Race Result

In [8]:
def build_features(seasons):
    # session = fastf1.get_session(season, gp_name, 'FP2')
    # session.load()
    
    practice_df = get_practice_data(seasons)
    quali_df =  get_qualifying_data(seasons)
    race_df = get_race_labels(seasons)

    # Merging the data
    df = practice_df.merge(quali_df, on='DriverID', how='left')
    df = df.merge(race_df, on='DriverID', how='left')

    return df

In [9]:
seasons = [2021, 2022, 2023, 2024]
features_df = build_features(seasons)
features_df

Season: 2021


events      WARNING 	Correcting user input 'Pre-Season Test' to 'United States Grand Prix'
core           INFO 	Loading data for United States Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Race: Pre-Season Test


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['3', '4', '5', '6', '7', '9', '10', '11', '14', '16', '18', '22', '31', '33', '44', '47', '55', '63', '77', '99']
events      WARNING 	Correcting user input 'Pre-Season Test' to 'United States Grand Prix'
core           INFO 	Loading data for United States Grand Prix - Practice 2 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req           

Race: Bahrain Grand Prix


...

core        WARNING 	Driver 63: Lap timing integrity check failed for 2 lap(s)
core        WARNING 	Driver 44: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 55: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 16: Lap timing integrity check failed for 2 lap(s)
core        WARNING 	Driver  1: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver  4: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 81: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 30: Lap timing integrity check failed for 2 lap(s)
core        WARNING 	Driver 77: Lap timing integrity check failed for 2 lap(s)
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 63 completed the race distance 

Race: Qatar Grand Prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '81', '63', '10', '55', '14', '24', '20', '4', '77', '44', '22', '30', '23', '27', '11', '18', '43', '31']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Race: Abu Dhabi Grand Prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '55', '16', '44', '63', '1', '10', '27', '14', '81', '23', '22', '24', '18', '61', '20', '30', '77', '43', '11']


In [27]:
df = practice_df.merge(quali_df, on=['Season', 'Race', 'DriverID'], how='left')
df = df.merge(race_df, on=['Season', 'Race', 'DriverID'], how='left')

In [29]:
df

,Season,Race,DriverID,FastestPracticeLap,AvgPracticeLap,Q1Time,Q2Time,Q3Time,GridPos,RacePosition
0,2021,Pre-Season Test,ricciardo,95.345,109.992026,94.407,94.643,93.808,7.0,NaN
1,2021,Pre-Season Test,norris,94.945,107.691345,94.551,93.880,93.887,8.0,NaN
2,2021,Pre-Season Test,vettel,95.851,106.818941,95.281,95.500,NaN,12.0,NaN
3,2021,Pre-Season Test,latifi,96.118,109.432850,95.995,NaN,NaN,17.0,NaN
4,2021,Pre-Season Test,raikkonen,96.062,112.151591,96.311,NaN,NaN,18.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1978,2024,Abu Dhabi Grand Prix,stroll,84.531,106.555658,83.729,83.784,NaN,13.0,14.0
1979,2024,Abu Dhabi Grand Prix,tsunoda,84.343,111.313000,83.735,83.419,NaN,11.0,12.0
1980,2024,Abu Dhabi Grand Prix,albon,84.269,105.476520,83.821,NaN,NaN,16.0,11.0
1981,2024,Abu Dhabi Grand Prix,sainz,83.871,100.196750,83.487,82.985,82.824,3.0,2.0


In [30]:
df.to_csv('final_data.csv', index=False)

In [31]:
df = pd.read_csv("final_data.csv", delimiter=",")

In [32]:
df.head()

,Season,Race,DriverID,FastestPracticeLap,AvgPracticeLap,Q1Time,Q2Time,Q3Time,GridPos,RacePosition
0,2021,Pre-Season Test,ricciardo,95.345,109.992026,94.407,94.643,93.808,7.0,NaN
1,2021,Pre-Season Test,norris,94.945,107.691345,94.551,93.880,93.887,8.0,NaN
2,2021,Pre-Season Test,vettel,95.851,106.818941,95.281,95.500,NaN,12.0,NaN
3,2021,Pre-Season Test,latifi,96.118,109.432850,95.995,NaN,NaN,17.0,NaN
4,2021,Pre-Season Test,raikkonen,96.062,112.151591,96.311,NaN,NaN,18.0,NaN


## Feature Engineering

In [35]:
df.head()

,Season,Race,DriverID,FastestPracticeLap,AvgPracticeLap,Q1Time,Q2Time,Q3Time,GridPos,RacePosition
20,2021,Bahrain Grand Prix,ricciardo,91.230,108.376614,90.795,90.222,89.927,6.0,7.0
21,2021,Bahrain Grand Prix,norris,90.942,110.350455,90.902,90.099,89.974,7.0,4.0
22,2021,Bahrain Grand Prix,vettel,91.769,108.599510,92.056,NaN,NaN,18.0,15.0
23,2021,Bahrain Grand Prix,latifi,93.400,108.627216,91.936,NaN,NaN,17.0,18.0
24,2021,Bahrain Grand Prix,raikkonen,91.862,108.040345,91.547,91.238,NaN,14.0,11.0


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1938 entries, 20 to 1982
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Season              1938 non-null   int64  
 1   Race                1938 non-null   object 
 2   DriverID            1857 non-null   object 
 3   FastestPracticeLap  1938 non-null   float64
 4   AvgPracticeLap      1938 non-null   float64
 5   Q1Time              1836 non-null   float64
 6   Q2Time              1377 non-null   float64
 7   Q3Time              906 non-null    float64
 8   GridPos             1856 non-null   float64
 9   RacePosition        1854 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 166.5+ KB


### 1 - Removing Pre-Season Test Rows

<b>Note</b>: 
- Since there are no preseason tests don't have practice session or qualifying sessions or final grid positions, including such rows will add no value and are not viable data points

In [33]:
df = df[df['Race'] != 'Pre-Season Test']

In [34]:
df.head()

,Season,Race,DriverID,FastestPracticeLap,AvgPracticeLap,Q1Time,Q2Time,Q3Time,GridPos,RacePosition
20,2021,Bahrain Grand Prix,ricciardo,91.230,108.376614,90.795,90.222,89.927,6.0,7.0
21,2021,Bahrain Grand Prix,norris,90.942,110.350455,90.902,90.099,89.974,7.0,4.0
22,2021,Bahrain Grand Prix,vettel,91.769,108.599510,92.056,NaN,NaN,18.0,15.0
23,2021,Bahrain Grand Prix,latifi,93.400,108.627216,91.936,NaN,NaN,17.0,18.0
24,2021,Bahrain Grand Prix,raikkonen,91.862,108.040345,91.547,91.238,NaN,14.0,11.0


### 2 - Inspecting missing driverID and Removing Nulls 

In [37]:
null_driverid_df = df[df['DriverID'].isnull()]
null_driverid_df

,Season,Race,DriverID,FastestPracticeLap,AvgPracticeLap,Q1Time,Q2Time,Q3Time,GridPos,RacePosition
79,2021,Portuguese Grand Prix,NaN,81.806,89.152667,NaN,NaN,NaN,NaN,NaN
95,2021,Spanish Grand Prix,NaN,80.700,92.586750,NaN,NaN,NaN,NaN,NaN
99,2021,Spanish Grand Prix,NaN,81.887,97.836556,NaN,NaN,NaN,NaN,NaN
158,2021,French Grand Prix,NaN,97.881,115.508000,NaN,NaN,NaN,NaN,NaN
182,2021,Styrian Grand Prix,NaN,67.823,88.532333,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1967,2024,Abu Dhabi Grand Prix,NaN,85.471,104.907938,NaN,NaN,NaN,NaN,NaN
1968,2024,Abu Dhabi Grand Prix,NaN,85.877,106.444842,NaN,NaN,NaN,NaN,NaN
1969,2024,Abu Dhabi Grand Prix,NaN,86.179,108.647053,NaN,NaN,NaN,NaN,NaN
1970,2024,Abu Dhabi Grand Prix,NaN,86.121,104.538286,NaN,NaN,NaN,NaN,NaN


In [38]:
null_driverid_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 79 to 1973
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Season              81 non-null     int64  
 1   Race                81 non-null     object 
 2   DriverID            0 non-null      object 
 3   FastestPracticeLap  81 non-null     float64
 4   AvgPracticeLap      81 non-null     float64
 5   Q1Time              0 non-null      float64
 6   Q2Time              0 non-null      float64
 7   Q3Time              0 non-null      float64
 8   GridPos             0 non-null      float64
 9   RacePosition        0 non-null      float64
dtypes: float64(7), int64(1), object(2)
memory usage: 7.0+ KB


<b>Observations:</b>
- The missing driverIDs have everything missing from Qualifying times, GridPositions and RacePositions
- This lack of driverid,  qualifying data and race data could attribute to constructors giving reserve drivers only practice sessions, which is quite common in F1
- Thus it is sensible to remove this missing data

In [39]:
df = df.dropna(subset=['DriverID'])

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1857 entries, 20 to 1982
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Season              1857 non-null   int64  
 1   Race                1857 non-null   object 
 2   DriverID            1857 non-null   object 
 3   FastestPracticeLap  1857 non-null   float64
 4   AvgPracticeLap      1857 non-null   float64
 5   Q1Time              1836 non-null   float64
 6   Q2Time              1377 non-null   float64
 7   Q3Time              906 non-null    float64
 8   GridPos             1856 non-null   float64
 9   RacePosition        1854 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 159.6+ KB


In [41]:
### 2 - Inspecting missing raceposition and filling in the missing Nulls 

In [42]:
null_racepos_df = df[df['RacePosition'].isnull()]

In [43]:
null_racepos_df

,Season,Race,DriverID,FastestPracticeLap,AvgPracticeLap,Q1Time,Q2Time,Q3Time,GridPos,RacePosition
455,2021,Abu Dhabi Grand Prix,mazepin,86.332,106.386891,85.685,NaN,NaN,20.0,NaN
552,2022,Saudi Arabian Grand Prix,mick_schumacher,90.765,107.550130,90.167,89.92,NaN,14.0,NaN
1293,2023,Singapore Grand Prix,stroll,93.126,109.927176,93.397,NaN,NaN,20.0,NaN


<b>Explanations</b>:
- Nikita Mazepin qualified 20th for the Abu Dhabi Grand Prix-2021 but he withdrew before the race as he tested positive for coronavirus. His place on the grid was left vacant
- Mick Schumacher qualified 14th for the Saudi Arabian Grand Prix-2022 but but his car was withdrawn following a crash, and he did not take the start. Drivers who qualified behind him gained a grid position as he officially did not progress beyond qualifying
- Lance Stroll crashed out in Q1 of Singapore Grand Prix-2023 and did not partake in the race
- Since this is a small set of data, we can remove these or fill the race positions as last, either way, they will not add any information
- On the contrary, adding a race position as 20 can hinder the model training as it's highly possible that the drivers can have good practice and quali sessions and a 20th position might not be the most appropriate results
- This it is feasible to remove the 